# Medluma - The AI-powered Disease Information Portal
## Project Overview
Medluma is a multi-agent app designed to provide a variety of information about any disease. This includes a description of the disease, current research, clinical studies and recent novel findings highlighted in the media. The app retreives data from a variety of sources and allows the user to choose a comprehensive (includes detailed biomedical and clinical information) or a simple (simiar to a short news article in a scientific journal) output format.

## Section 1: Setup

In [1]:
# Configure Gemini API keys
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
# Install BioMCP python package
!pip install biomcp-python

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.0/175.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 90.5 MB/s eta 0:00:00:00:0100:01
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=beafa4f5041fe8d9648e99ca7d67a424be3f3ba6d6ea08f3cbb314009607c12e
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built intervaltree


In [3]:
# Import python packages
import shutil
import sys
import warnings
import logging
import uuid
import os

# Suppress all warnings
warnings.filterwarnings('ignore')
logging.getLogger('asyncio').setLevel(logging.CRITICAL)
logging.getLogger('google_genai.types').setLevel(logging.ERROR)
logging.getLogger('google.adk').setLevel(logging.ERROR)

# Gemini packages
from google.genai import types

# MCP and ADK packages
from mcp import StdioServerParameters
from google.adk.agents import LlmAgent, Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools import AgentTool, FunctionTool, google_search

print("✅ Components imported successfully.")

✅ Components imported successfully.


In [4]:
# from google.adk.tools import ToolContext
# from google.adk.apps.app import App, ResumabilityConfig
# from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
# from google.adk.runners import Runner
# from google.adk.sessions import InMemorySessionService
# from google.adk.tools import FunctionTool
# from google.adk.models.google_llm import Gemini
# import uuid
# import shutil
# import sys
# import os

In [5]:
# Setup retry configuration
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print(f"✅ Retry configuration setp")

✅ Retry configuration setp


## Section 2: Tools

### MCP Tools

In [6]:
# Locate biomcp and configure BioMCP mcp tool
biomcp_path = shutil.which("biomcp")
if not biomcp_path:
    possible_paths = [
        os.path.join(sys.prefix, "bin", "biomcp"),
        os.path.join(os.path.dirname(sys.executable), "biomcp"),
        os.path.expanduser("~/.local/bin/biomcp")
    ]
    for path in possible_paths:
        if os.path.exists(path):
            biomcp_path = path
            break

    if biomcp_path:
        print(f"Using BioMCP at: {biomcp_path}")

    else:
        print("⚠️ biomcp not found - run !pip install biomcp-python")

print(f"✅ BioMCP loaded and located in path: {biomcp_path}")

✅ BioMCP loaded and located in path: /usr/local/bin/biomcp


In [7]:
# Configure BioMCP MCP tool
mcp_bio_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command=biomcp_path,
            args=["run"],
            env={"MCP_LOG_LEVEL": "debug"}
        ),
        timeout=120,
    )
)

print(f"✅ Created BioMCP mcp tool")

✅ Created BioMCP mcp tool


### Other Tools

In [8]:
# Pausable preference function tool
def get_output_preference(tool_context: ToolContext) -> dict:
    """Ask user for output preference."""
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint="Would you like 'comprehensive' (detailed summaries + references) or 'simple' (article only) output?",
            payload={"preference_type": "output_format"}
        )
        return {
            "status": "pending",
            "message": "Waiting for user preference..."
        }
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "confirmed",
            "message": "User preference received."
        }
    else:
        return {
            "status": "rejected",
            "message": "User cancelled."
        }


def exit_loop():
    return {"status": "approved", "message": "Article approved."}

print(f"✅ Define Get_output_preference and exit_loop functions")

✅ Define Get_output_preference and exit_loop functions


In [9]:
# Helper function (check for approval)
def check_for_approval(events):
    """Check if events contain approval request."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (part.function_call and
                    part.function_call.name == "adk_request_confirmation"):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

print(f"✅ Helper function created")

✅ Helper function created


## Section 3: Agents

In [10]:
# Define all agents

# Coordinator Agent
coordinator_agent = Agent(
    name="CoordinatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Call get_output_preference tool. After user responds, check their message:
    - If 'comprehensive': output "comprehensive"
    - If 'simple': output "simple"
    - Default: "simple"
    Output ONLY the preference word.""",
    tools=[FunctionTool(func=get_output_preference)],
    output_key="user_preference",
)
print("✅ coordinator_agent created.")


# Biomedical researcher Agent: Search various online databases for information on clinical trials and current research for a particular disease or research area.
bio_researcher = Agent(
    name="bio_researcher",
    model=Gemini(
        model="gemini-2.5-flash", 
        retry_options=retry_config
    ), 
    instruction="You are a researcher. Use the mcp tool to find research and clinical trial information. Include known mutations associated with this disease. Only output a brief summary with appropriate references.",
    tools=[mcp_bio_server],
    output_key="bio_research",
)
print("✅ bio_researcher created.")


# Health Researcher Agent: Performs google search focusing on medical breakthroughs for a particular disease or research area.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs for a particular disease or research area. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words). Include relevant references.""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")


# Aggregator Agent
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Combine these findings into an executive summary:
    **Research:** {bio_research}
    **News:** {health_research}
    Highlight key takeaways (200 words).""",
    output_key="executive_summary",
)
print("✅ aggregator_agent created.")


# Scientific article writer agent
initial_science_writer_agent = Agent(
    name="InitialScienceWriterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on: {executive_summary}, write a first draft article (100-150 words).
    Output only the article text.""",
    output_key="current_science_article",
)
print("✅ initial_science_writer_agent created.")


# Scientific and article critique agent
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Review: {current_science_article}
    If well-written with references: respond "APPROVED"
    Otherwise: provide 2-3 suggestions.""",
    output_key="critique",
)
print("✅ critic_agent created.")


# Article refiner agent
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Draft: {current_science_article}
    Critique: {critique}
    If critique is "APPROVED": call exit_loop
    Otherwise: rewrite incorporating feedback.""",
    output_key="current_science_article",
    tools=[FunctionTool(exit_loop)],
)
print("✅ refiner_agent created.")


# Final output agent
final_output_agent = Agent(
    name="FinalOutputAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on: {user_preference}
    
    If COMPREHENSIVE:
    **EXECUTIVE SUMMARY**
    {executive_summary}

    **KEY DEVELOPMENTS**
    {health_research} (2-3 points only)
    
    If SIMPLE:
    {current_science_article}""",
    output_key="final_output",
)
print("✅ final_output_agent created.")

✅ coordinator_agent created.
✅ bio_researcher created.
✅ health_researcher created.
✅ aggregator_agent created.
✅ initial_science_writer_agent created.
✅ critic_agent created.
✅ refiner_agent created.
✅ final_output_agent created.


## Section 4: Pipeline

In [11]:
# Pipeline Construction

# Article refinement loop
article_refinement_loop = LoopAgent(
    name="ArticleRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,
)

# Research pipeline
research_pipeline = SequentialAgent(
    name="ResearchPipeline",
    sub_agents=[
        bio_researcher,      # Run FIRST (before pause)
        health_researcher,   # Run SECOND (before pause)
    ],
)

# Root Agent to orchestrate agent flow
test_root_agent = SequentialAgent(
    name="TestPipeline",
    sub_agents=[
        coordinator_agent,              # THEN ask for preference (pause here)
        research_pipeline,              # Do all research FIRST
        aggregator_agent,               # Resume: aggregate results
        initial_science_writer_agent,   # Write article
        article_refinement_loop,        # Refine
        final_output_agent,             # Format output
    ],
)

print(f"✅ Pipeline constructed")

✅ Pipeline constructed


## Section 5: Runner

In [12]:
# App and Runner setup
session_service = InMemorySessionService()

test_app = App(
    name="article_coordinator_test",
    root_agent=test_root_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

test_runner = Runner(
    app=test_app,
    session_service=session_service,
)

print(f"✅ App and Runner configured")

✅ App and Runner configured


## Section 6: Workflow orchestration function

In [13]:
# Workflow

async def run_test_workflow(query: str):
    """Run workflow with user interaction."""
    
    print(f"\n{'='*60}")
    print(f"User > {query}\n")
    
    session_id = f"test_{uuid.uuid4().hex[:8]}"
    
    await session_service.create_session(
        app_name="article_coordinator_test",
        user_id="test_user",
        session_id=session_id
    )
    
    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []
    
    print("🔄 Starting...")
    async for event in test_runner.run_async(
        user_id="test_user",
        session_id=session_id,
        new_message=query_content
    ):
        events.append(event)
    
    approval_info = check_for_approval(events)
    
    if approval_info:
        print(f"⏸️  Pausing for preference...\n")
        
        user_choice = input("Your choice (comprehensive/simple): ").strip()
        print(f"\n✅ You selected: {user_choice}\n")
        
        confirmation_response = types.FunctionResponse(
            id=approval_info["approval_id"],
            name="adk_request_confirmation",
            response={"confirmed": True},
        )
        
        combined_content = types.Content(
            role="user",
            parts=[
                types.Part(function_response=confirmation_response),
                types.Part(text=user_choice)
            ]
        )
        
        print("🔄 Resuming...")
        async for event in test_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=combined_content,
            invocation_id=approval_info["invocation_id"],
        ):
            pass  # Just process
        
        print(f"✅ Completed\n")
    
    # Display output
    session = await session_service.get_session(
    app_name="article_coordinator_test",  # Use the actual app name
    user_id="test_user",                  # Use the actual user ID
    session_id=session_id
    )
    if 'final_output' in session.state:
        print("\n" + "="*60)
        print("FINAL OUTPUT:")
        print("="*60)
        print(session.state['final_output'])
    else:
        print(f"\nState keys: {list(session.state.keys())}")

print("✅ Workflow ready")

✅ Workflow ready


## Section 7: Query input and run workflow

In [14]:
# Run
await run_test_workflow(
    "Write an article on current advances in the treatment of multiple sclerosis"
)


User > Write an article on current advances in the treatment of multiple sclerosis

🔄 Starting...
⏸️  Pausing for preference...



Your choice (comprehensive/simple):  simple



✅ You selected: simple

🔄 Resuming...
✅ Completed


FINAL OUTPUT:
Multiple Sclerosis (MS) is a chronic disease affecting the central nervous system. Recent advances in treatment have focused on slowing disease progression, managing symptoms, and improving the quality of life for those affected.

**Disease-Modifying Therapies (DMTs):**

A significant area of advancement lies in DMTs, which aim to reduce the frequency and severity of relapses and slow the accumulation of disability. New DMTs with novel mechanisms of action continue to emerge, offering more personalized treatment options. These include:

*   **High-efficacy therapies:** These treatments, often administered intravenously, target specific immune cells or pathways involved in the inflammatory process of MS. They have shown remarkable effectiveness in reducing relapses and MRI activity.
*   **Oral medications:** The development of oral DMTs has improved convenience and adherence for many patients. These medications offer var